# COVID-19 CT Image Classification Using PyTorch



## Steps

1. Data Preparation
2. Initial Model Development
3. Model Performance Using Data Augmentation
4. Transfer Learning

In [ ]:
import os  
import glob
import sklearn
from sklearn.model_selection import train_test_split

import PIL 
import numpy as np
import matplotlib.pyplot as plt 

import torch
import torch.nn as nn
# from torchinfo import summary 

import torch.optim as optim
from IPython.display import Image
from torch.utils.data import DataLoader, Dataset

from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms

import cv2

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device 

Set Random Seed for reproducability

In [ ]:
random_seed = 124
np.random.seed(random_seed)

torch.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True

### 1.2 Generating Labels and Creating Sets for Modeling

Extract file links for both postive and negative images and split the dataset into train, validation, and test sets

In [ ]:
path = '/kaggle/input/covidct/'

pos_files = glob.glob(os.path.join(path, "CT_COVID",'*.*'))
neg_files = glob.glob(os.path.join(path, 'CT_NonCOVID','*.*'))

images = pos_files + neg_files
labels = np.array([1]*len(pos_files)+[0]*len(neg_files))

images_tv, images_test, y_tv, y_test  = train_test_split(images, labels, shuffle=True, test_size=0.2, random_state=123)
images_train, images_val, y_train, y_val  = train_test_split(images_tv, y_tv, shuffle=True, test_size=0.25, random_state=123)

From the plot below, observe that we have a balanced yet very small dataset at our disposal. Not ideal, but never the end of the road. Let's do our best to make the most out of what we have before we collect more data.

In [ ]:
num_pos, num_neg = len(pos_files), len(neg_files)

plt.title('Distribution of labels')
plt.bar(['Positive', 'Negative'], [num_pos, num_neg])
plt.show()

Let's take a look at some of the images!

In [ ]:
Image(images_train[1])
Image(images_train[15])
Image(images_train[66])

In [ ]:
im = [cv2.imread(images_train[i]) for i in range(6)]

fig,ax = plt.subplots(ncols=6, figsize=(18,6))
for i in range(len(im)):
    ax[i].imshow(im[i],cmap='gray')

plt.show()

In [ ]:
print(f'Number of samples in each set (train, val, test): {len(y_train), len(y_val), len(y_test)}')

print(f'Number of positive samples in each set: {y_train.sum(), y_val.sum(), y_test.sum()}')

### Dataset Class



In [ ]:
class CT_Dataset(Dataset):
    def __init__(self, img_path, img_labels, img_transforms=None, grayscale=True):
        self.img_path = img_path
        self.img_labels = torch.Tensor(img_labels)
        if (img_transforms is None) & (grayscale == True):
            self.transforms = transforms.Compose([transforms.Grayscale(),
                                                  transforms.Resize((250, 250)),
                                                  transforms.ToTensor()])
        elif grayscale == False:
            self.transforms = transforms.Compose([transforms.Resize((250, 250)),
                                                  transforms.ToTensor()])
        else:
            self.transforms = img_transforms
    
    def __getitem__(self, index):
        # load image
        cur_path = self.img_path[index]
        cur_img = PIL.Image.open(cur_path).convert('RGB')
        cur_img = self.transforms(cur_img)

        return cur_img, self.img_labels[index]
    
    def __len__(self):
        return len(self.img_path)

## 2. Model Development

### 2.1 Network Configuration

The model used for this exercise has several key layers:


*   Conv2D $\rightarrow$ Batch Normalization $\rightarrow$ ReLU Activation $\rightarrow$ AvgPooling2D
  * Output Channels: 64
  * Filter Size: 3x3
  * AvgPooling Filter Size: 2x2
*   Conv2D $\rightarrow$ Batch Normalization $\rightarrow$ ReLU Activation $\rightarrow$ AvgPooling2D
  * Output Channels: 128
  * Filter Size: 3x3
  * AvgPooling Filter Size: 2x2
*   Conv2D $\rightarrow$ Batch Normalization $\rightarrow$ ReLU Activation $\rightarrow$ AvgPooling2D
  * Output Channels: 256
  * Filter Size: 3x3
  * AvgPooling Filter Size: 2x2
*   Conv2D $\rightarrow$ Batch Normalization $\rightarrow$ ReLU Activation $\rightarrow$ AvgPooling2D
  * Output Channels: 512
  * Filter Size: 3x3
  * AvgPooling Filter Size: 2x2
*   Conv2D $\rightarrow$ Batch Normalization $\rightarrow$ ReLU Activation $\rightarrow$ AvgPooling2D $\rightarrow$ Flatten
  * Output Channels: 512
  * Filter Size: 3x3
  * AvgPooling Filter Size: 2x2
  * Output Shape (After Flatten): (Batch Size, 1600)

After the series of convolutions, we pass the parameters through a series of Dropout, Linear, and ReLU layers and produce an output of shape (Batch Size, 1)
*   Dropout $\rightarrow$ Linear $\rightarrow$ ReLU Activation $\rightarrow$ Dropout $\rightarrow$ Linear $\rightarrow$ ReLU Activation $\rightarrow$ Linear $\rightarrow$ ReLU Activation $\rightarrow$ Linear
  * Dropout Ratio: 0.5

In [ ]:
# define CNN mode
class Convnet(nn.Module):
    
    def __init__(self, dropout=0.5):
        super(Convnet, self).__init__()
        self.convnet = nn.Sequential(
          # input (num_batch, 1, 250, 250)
          nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3),  # (num_batch, 64, 248, 248)
          nn.BatchNorm2d(64),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2),  # (num_batch, 64, 124, 124)

          nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3), # (num_batch, 128, 122, 122)
          nn.BatchNorm2d(128),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2),  # (num_batch, 128, 61, 61)

          nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3), # (num_batch, 256, 59, 59)
          nn.BatchNorm2d(256),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2),  # (num_batch, 256, 29, 29)

          nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3), # (num_batch, 128, 27, 27)
          nn.BatchNorm2d(512),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2),  # (num_batch, 128, 13, 13)

          nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3), # (num_batch, 64, 11, 11)
          nn.BatchNorm2d(512),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2),  # (num_batch, 64, 5, 5)
          nn.Flatten() # (num_batch, 1600)
        )
        self.classifier = nn.Sequential(
            nn.Dropout(dropout),  # Dropout before first linear layer since it has a large number of trainable parameters
            nn.Linear(in_features= 12800, out_features=512),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(in_features=512, out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=1)
        )
    def forward(self, x):
        x = self.convnet(x)
        x = self.classifier(x)
        return x

#### Print model summary to  visualize network structure

In [ ]:
vision_model = Convnet()
summary(vision_model, (32, 1, 250, 250))

### 2.2 Model Training Procedure
The training sequence used for our CNN model is summarized below:


*   **Loss Function**: *Binary Cross Entropy w/ Logistic Loss*
*   **Optimizer**: *Adam Optimization*
  * To fight overfitting the following methods were used:
      * **L2 Regularization**: Weight regularization using L2 Norm
      * **Learning Schedule**: *Decrease the learning rate* over a set period of epochs

Parameters used for training:


*   **Initial Learning Rate**: 0.0002
*   Learning Schedule: 
  * **Gamma**: 0.5
  * **Patience**: 7 epochs
*   **Number of Epochs**: 35
*   **Batch Size**: 32
*   **L2 Weight Decay**: 0.09


In [ ]:
# define training function

def train_model(model, train_dataset, val_dataset, test_dataset, device, 
                lr=0.0001, epochs=30, batch_size=32, l2=0.00001, gamma=0.5,
                patience=7):
    model = model.to(device)

    # construct dataloader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    # history
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

    # set up loss function and optimizer
    criterion = nn.BCEWithLogitsLoss()  
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=l2)  # pass in the parameters to be updated and learning rate
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=patience, gamma=gamma)

    # Training Loop
    print("Training Start:")
    for epoch in range(epochs):
        model.train()  # start to train the model, activate training behavior

        train_loss = 0
        train_acc = 0
        val_loss = 0
        val_acc = 0

        for i, (images, labels) in enumerate(train_loader):
            # reshape images
            images = images.to(device)  # reshape: from (128, 1, 28, 28) -> (128, 28 * 28) = (128, 284), move batch to device
            labels = labels.to(device)  # move to device
            # forward
            outputs = model(images).view(-1)  # forward
            pred = torch.sigmoid(outputs)
            pred = torch.round(pred)
    
            cur_train_loss = criterion(outputs, labels)  # loss
            cur_train_acc = (pred == labels).sum().item() / batch_size

            # backward
            cur_train_loss.backward()   # run back propagation
            optimizer.step()            # optimizer update all model parameters
            optimizer.zero_grad()       # set gradient to zero, avoid gradient accumulating

            # loss
            train_loss += cur_train_loss 
            train_acc += cur_train_acc
        
        # valid
        model.eval()  # start to train the model, activate training behavior
        with torch.no_grad():  # tell pytorch not to update parameters
            for images, labels in val_loader:
                # calculate validation loss
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images).view(-1)

                # loss
                cur_valid_loss = criterion(outputs, labels)
                val_loss += cur_valid_loss
                # acc
                pred = torch.sigmoid(outputs)
                pred = torch.round(pred)
                val_acc += (pred == labels).sum().item() / batch_size

        # learning schedule step
        scheduler.step()

        # print training feedback
        train_loss = train_loss / len(train_loader)
        train_acc = train_acc / len(train_loader)
        val_loss = val_loss / len(val_loader)
        val_acc = val_acc / len(val_loader)

        print(f"Epoch:{epoch + 1} / {epochs}, lr: {optimizer.param_groups[0]['lr']:.5f} train loss:{train_loss:.5f}, train acc: {train_acc:.5f}, valid loss:{val_loss:.5f}, valid acc:{val_acc:.5f}")
    
        # update history
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
    
    test_acc = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            # calculate outputs by running images through the network
            outputs = model(images)

            # the class with the highest energy is what we choose as prediction
            pred = torch.sigmoid(outputs)
            pred = torch.round(pred)
            test_acc += (pred == labels).sum().item()

    print(f'Test Accuracy:  {(test_acc / len(test_loader))}')

    return history

Process the datasets and train the model

In [ ]:
# Load the data
train_dataset = CT_Dataset(img_path=images_train, img_labels=y_train)
val_dataset = CT_Dataset(img_path=images_val, img_labels=y_val)
test_dataset = CT_Dataset(img_path=images_test, img_labels=y_test)

# Train the CNN model
cnn_model = Convnet(dropout=0.5)
hist = train_model(cnn_model, train_dataset, val_dataset, test_dataset, device, lr=0.0002, batch_size=32, epochs=35, l2=0.09, patience=5)

In [ ]:
# plot training curves
epochs = range(1, len(hist['train_loss']) + 1)

fig, ax = plt.subplots(1,2, figsize=(18,6))
ax[0].plot(epochs, hist['train_loss'], 'r-', label='Train')
ax[0].plot(epochs, hist['val_loss'], 'b-', label='Evaluation')
ax[0].set_title('Loss')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Loss')
ax[0].legend()

ax[1].plot(epochs, hist['train_acc'], 'r-', label='Train')
ax[1].plot(epochs, hist['val_acc'], 'b-', label='Evaluation')
ax[1].set_title('Accuracy')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Acc')
ax[1].legend()

plt.show()

#### Results For Initial Model

The learning curves shown above show reasonable results.

**Observations** 


*   The model begins to overfit on the training data after 10 epochs
*   Training Accuracy generally reaches around 99\% relatively quickly
* Validation accuracy stabilizes at around 80\%.
* Training loss converges to near zero
* Validation loss converges around 0.4
* Accuracy on the test set is 88\% -- a surprising result given the validation performance

Overall, our first model achieved an accuracy of 88\% for our hold out set. Given the size of our network and the lack of available data, these results are quite satisfying. With that being said, we cannot ignore the fact that our model overfits quite early in training. Since we achieve a near perfect score for training, I have no doubt we can continue to tune this model and achieve even better results on our hold-out set and bridge the gap between performance for training and validation.

**Considerations to fight overfitting**


*   Add L1 Regularization
*   Data Augmentation (see next section)
*   Transfer Learning (see later sections)
*   More hyperparameter tuning

## 3. Data Augmentation

Data Augmentation is a technique used to either diversify a training set or add more samples to it. To augment an image is to apply specified transformation to the pixels of the image array. Examples of augmentations include rescaling, resizing, and translating the image to name a few. Implementing these augmentationss in PyTorch is quite straightforward. In this section we will apply augmentations on the entire dataset as well as a fraction of our data to generate "new" samples to add to our dataset and run our model to compare performance.

#### Augmentations Used
* **Grayscale**
* **Resize** $\rightarrow$ 250 x 250
* **RandomAffine**
  * **Translate**: Width $\rightarrow$ 0.01, Height $\rightarrow$ 0.001
  * **Image Scaling**: Width $\rightarrow$ 1.2x, Height $\rightarrow$ 1.2x
  * **Shear**: 0.9
* **RandomRotation**: 20 degrees
* **Convert Image to Tensor**

In [ ]:
img = PIL.Image.open(images_train[10])

img_trans = transforms.Compose([transforms.Grayscale(),
                                transforms.RandomRotation(5),
                                transforms.Resize((250, 250)),
                                transforms.RandomAffine(degrees=0, scale=(1.1, 1.1), shear=0.9),
                                transforms.ToTensor()
                                ])
trans = img_trans(img)

print('Before Transformation')
display(img)
print('\nAfter Transformation')
display(transforms.ToPILImage()(trans))

### 3.1 Data Augmentation Approach \#1 -- Augmenting the entire Training Set

In [ ]:
train_dataset_full_aug = CT_Dataset(img_path=images_train, img_labels=y_train, img_transforms=img_trans)
val_dataset = CT_Dataset(img_path=images_val, img_labels=y_val)
test_dataset = CT_Dataset(img_path=images_test, img_labels=y_test)

In [ ]:
# Train the CNN model
cnn_model = Convnet()
hist_full_aug = train_model(cnn_model, train_dataset_full_aug, val_dataset, 
                            test_dataset, device, lr=0.0001, batch_size=32, epochs=35,
                            gamma=0.75, l2=0.09, patience=15)

In [ ]:
# plot training curves
epochs = range(1, len(hist_full_aug['train_loss']) + 1)

fig, ax = plt.subplots(1,2, figsize=(20,6))
ax[0].plot(epochs, hist_full_aug['train_loss'], 'r-', label='Train')
ax[0].plot(epochs, hist_full_aug['val_loss'], 'b-', label='Evaluation')
ax[0].set_title('Loss')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Loss')
ax[0].legend()

ax[1].plot(epochs, hist_full_aug['train_acc'], 'r-', label='Train')
ax[1].plot(epochs, hist_full_aug['val_acc'], 'b-', label='Evaluation')
ax[1].set_title('Accuracy')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Acc')
ax[1].legend()

plt.show()

#### Results from Approach 1

**Observations**

*   Drop in overall accuracy for both training and validation sets
*   Test accuracy came out to be 78\% -- a considerable drop compared to the initial model 
* The drop in overall performance may suggest that perhaps lack of data could be a contributing factor rather than the diversity in available samples

### 3.2 Data Augmentation Approach \#2 -- Concatenate Augmented Data to Original Dataset
In this approach, the aim is to see if adding augmented samples into our dataset can help fight overfitting. We know that our sample size is small, so perhaps adding variations of samples on top of our original data may allow our model to generalize better. In doing this, we also run the risk of increasing the bias of our model.


In the code below we will augment 60 images from the training set and 20 images from the validation set and concatenate them on the original data. 

In [ ]:
train_dataset_og = CT_Dataset(img_path=images_train, img_labels=y_train)
train_dataset_aug = CT_Dataset(img_path=images_train[:60], img_labels=y_train[:60], img_transforms=img_trans)
train_dataset_fin = torch.utils.data.ConcatDataset([train_dataset_og,train_dataset_aug])

val_dataset_og = CT_Dataset(img_path=images_val, img_labels=y_val)
val_dataset_aug = CT_Dataset(img_path=images_val[:20], img_labels=y_val[:25], img_transforms=img_trans)
val_dataset_fin = torch.utils.data.ConcatDataset([val_dataset_og, val_dataset_aug])

test_dataset = CT_Dataset(img_path=images_test, img_labels=y_test)

In [ ]:
print(len(train_dataset_fin))
print(len(val_dataset_fin))

In [ ]:
# Train the CNN model
cnn_model = Convnet(dropout=0.5)
hist_concat = train_model(cnn_model, train_dataset_fin, val_dataset_fin, test_dataset, device, lr=0.0001, l2=0.09, batch_size=32, epochs=75)

In [ ]:
# plot training curves
epochs = range(1, len(hist_concat['train_loss']) + 1)

fig, ax = plt.subplots(1,2, figsize=(20,6))
ax[0].plot(epochs, hist_concat['train_loss'], 'r-', label='Train')
ax[0].plot(epochs, hist_concat['val_loss'], 'b-', label='Evaluation')
ax[0].set_title('Loss')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Loss')
ax[0].legend()


ax[1].plot(epochs, hist_concat['train_acc'], 'r-', label='Train')
ax[1].plot(epochs, hist_concat['val_acc'], 'b-', label='Evaluation')
ax[1].set_title('Accuracy')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Acc')
ax[1].legend()

plt.show()

#### Results from Approach 2 
**Observations**


*   Test Accuracy: 87\%
*   Distinct gap between training and validation results remain as learning plateaus after 20 epochs
*   The model achieves the highest accuracy for the hold out set compared to the previous models
*   Further investigation is needed to reduce the gap between training in validation sets 
*   It would be interesting to see how the model reacts to adding a greater portion of augmented samples on top of the original dataset

## 4. Transfer Learning (VGG-16)

In this section, we will use the VGG-16 model which is pretrained on Image-Net to leverage the pre-trained parameters for this particular classification task. To achieve this, we will load the vgg-16 model using the torchvision library and then freeze the parameters for the "features" portiion of the model. We will then create our own custom "classifier" sequence of dense layers to overwrite the pre-trained classifier parameters trained on Image-Net. In this way, we leverage the visual power of VGG-16 while still training the model to classify positive or negative instances of COVID-19 in CT imgaes.

In [ ]:
import torchvision.models as models

VGG_model = models.vgg16(pretrained=True)

In [ ]:
print(VGG_model)

In [ ]:
VGG_model = models.vgg16(pretrained=True)

for name, param in VGG_model.named_parameters():
    param.requires_grad = False

# define out classifier
binary_classifier = nn.Sequential(
   nn.Linear(in_features=25088, out_features=2048),
   nn.ReLU(),
   nn.Linear(in_features=2048, out_features=1024),
   nn.ReLU(),
   nn.Linear(in_features=1024, out_features=512),
   nn.ReLU(),
   nn.Linear(in_features=512, out_features=1)
)

# replace model class classifier attribute:
VGG_model.classifier = binary_classifier

In [ ]:
train_dataset = CT_Dataset(img_path=images_train, img_labels=y_train, grayscale=False)
val_dataset = CT_Dataset(img_path=images_val, img_labels=y_val, grayscale=False)
test_dataset = CT_Dataset(img_path=images_test, img_labels=y_test, grayscale=False)

# Train the CNN model
hist = train_model(VGG_model, train_dataset, val_dataset, test_dataset, device, lr=0.0001, batch_size=32, epochs=20, l2=0.2
                   , patience=15)

In [ ]:
# plot training curves
epochs = range(1, len(hist['train_loss']) + 1)

fig, ax = plt.subplots(1,2, figsize=(20,6))
ax[0].plot(epochs, hist['train_loss'], 'r-', label='Train')
ax[0].plot(epochs, hist['val_loss'], 'b-', label='Evaluation')
ax[0].set_title('Loss')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Loss')
ax[0].legend()


ax[1].plot(epochs, hist['train_acc'], 'r-', label='Train')
ax[1].plot(epochs, hist['val_acc'], 'b-', label='Evaluation')
ax[1].set_title('Accuracy')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Acc')
ax[1].legend()

plt.show()

### Results from Transfer Learning

**Observations**:



*   Learning curves remain stable, however the model overfits on the training set after 7 epochs
*   Training loss converges to nearly 0.1 while validation loss plateaus at around 0.4 
*   Validation accuracy peaks at 0.81 and converges to around 0.8 at the end of training
*   Results on the test set yield an accuracy of 84\% which beats our initial model

Overall, the use of transfer learning for image classification is quite beneficial. VGG-16 provided us with just as good results even though the convolutional layers were trained on a completely different dataset. This showcases the power of transfer learning as we freezed the weights for the convolutional layers for this task. 